In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('/home/prasun/impli/Iris.csv')
X = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
y = pd.get_dummies(df['Species']).values  # One-hot encode target

In [3]:
df.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Linear Regression

In [6]:
class SimpleLinearRegression:
    def fit(self, X, y):
        # Add bias term and solve using normal equation
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        self.theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
        return self

    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b.dot(self.theta)

# Train model
model = SimpleLinearRegression()
model.fit(X_train, y_train)

# Predict and get class labels
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test_labels)
print(f"Accuracy: {accuracy:.4f}")


[1 0 2 2 1 0 2 2 1 1 2 0 0 0 0 2 2 1 1 2 0 2 0 2 2 2 1 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy: 0.8667


Logistic Regression

In [26]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=1000, fit_intercept=True):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.theta = None
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.c_[np.ones((X.shape[0], 1)), X]
        
        self.theta = np.zeros(X.shape[1])
        
        for _ in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self._sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
        
        return self
    
    def predict_proba(self, X):
        if self.fit_intercept:
            X = np.c_[np.ones((X.shape[0], 1)), X]
        return self._sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold

# Train model using LogisticRegression
logistic_models = []
y_pred_prob = np.zeros((X_test.shape[0], y_test.shape[1]))

# Train one model per class (One-vs-All)
for i in range(y_train.shape[1]):
    model = LogisticRegression(lr=0.1, num_iter=1000)
    model.fit(X_train, y_train[:, i])
    logistic_models.append(model)
    y_pred_prob[:, i] = model.predict_proba(X_test)

# Get predictions
y_pred = np.argmax(y_pred_prob, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test_labels)
print(f"Accuracy: {accuracy:.4f}")

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy: 1.0000


Newton's Logistic Regression

In [27]:
class NewtonLogisticRegression:
    def __init__(self, num_iter=10, tol=1e-4):
        self.num_iter = num_iter
        self.tol = tol
        self.theta = None
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        # Add bias term
        X = np.c_[np.ones((X.shape[0], 1)), X]
        n_samples, n_features = X.shape
        self.theta = np.zeros(n_features)
        
        for _ in range(self.num_iter):
            # Compute sigmoid predictions
            z = np.dot(X, self.theta)
            h = self._sigmoid(z)
            
            # Compute gradient
            gradient = np.dot(X.T, (h - y)) / n_samples
            
            # Compute Hessian
            W = np.diag(h * (1 - h))
            hessian = (X.T @ W @ X) / n_samples
            
            # Newton step
            try:
                delta = np.linalg.solve(hessian, gradient)
            except np.linalg.LinAlgError:
                # Add small constant to diagonal if hessian is singular
                delta = np.linalg.solve(hessian + 1e-5 * np.eye(n_features), gradient)
            
            # Update parameters
            self.theta -= delta
            
            # Check convergence
            if np.sum(np.abs(delta)) < self.tol:
                break
        
        return self
    
    def predict_proba(self, X):
        X = np.c_[np.ones((X.shape[0], 1)), X]
        return self._sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold

# Train model using Newton's method
logistic_models = []
y_pred_prob = np.zeros((X_test.shape[0], y_test.shape[1]))

# Train one model per class (One-vs-All)
for i in range(y_train.shape[1]):
    model = NewtonLogisticRegression(num_iter=10)  # Fewer iterations needed for Newton's method
    model.fit(X_train, y_train[:, i])
    logistic_models.append(model)
    y_pred_prob[:, i] = model.predict_proba(X_test)

# Get predictions
y_pred = np.argmax(y_pred_prob, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test_labels)
print(f"Accuracy: {accuracy:.4f}")

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy: 1.0000


K-Nearest Neighbours

In [13]:
class KNN:
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self
    
    def predict(self, X):
        predictions = []
        for x in X:
            # Calculate distances to all training points
            distances = np.sqrt(np.sum((self.X_train - x)**2, axis=1))
            # Get k nearest neighbor indices
            k_indices = np.argsort(distances)[:self.k]
            # Get their labels
            k_nearest_labels = self.y_train[k_indices]
            # Predict majority class
            most_common = np.bincount(k_nearest_labels).argmax()
            predictions.append(most_common)
        return np.array(predictions)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = X_scaled[:-30]
X_test_scaled = X_scaled[-30:]

# Convert one-hot encoded y to class labels
y_train_labels = np.argmax(y[:-30], axis=1)
y_test_true = np.argmax(y[-30:], axis=1)

# Train KNN model
model = KNN(k=3)
model.fit(X_train_scaled, y_train_labels)

# Get predictions
y_pred = model.predict(X_test_scaled)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test_true)
print(f"KNN Accuracy: {accuracy:.4f}")

[2 2 2 1 2 2 2 1 2 2 2 2 2 1 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
KNN Accuracy: 0.8000


Random Forest

In [29]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=None, max_features='sqrt'):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []
        
    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]
    
    def fit(self, X, y):
        self.trees = []
        n_features = X.shape[1]
        
        if self.max_features == 'sqrt':
            self.feature_subset_size = int(np.sqrt(n_features))
        else:
            self.feature_subset_size = n_features
            
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            feature_idxs = np.random.choice(n_features, self.feature_subset_size, replace=False)
            tree.fit(X_sample[:, feature_idxs], y_sample)
            self.trees.append((feature_idxs, tree))
            
        return self
    
    def predict(self, X):
        predictions = np.array([tree.predict(X[:, feature_idxs]) 
                              for feature_idxs, tree in self.trees])
        return np.array([np.bincount(pred_row).argmax() 
                        for pred_row in predictions.T])

# Scale the data and prepare labels
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_labels = np.argmax(y, axis=1)

# Split the scaled data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_labels, test_size=0.2, random_state=42)

# Train Random Forest model
model = RandomForest(n_trees=100, max_depth=5)
model.fit(X_train, y_train)

# Get predictions
y_pred = model.predict(X_test)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Random Forest Accuracy: {accuracy:.4f}")

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Random Forest Accuracy: 1.0000


Decsion Tree

In [30]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        
    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)
        
    def _split(self, X, y, feature, threshold):
        left_mask = X[:, feature] <= threshold
        return (X[left_mask], y[left_mask], X[~left_mask], y[~left_mask])
    
    def _best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None
        
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                    
                gain = self._gini(y) - (len(y_left) * self._gini(y_left) + 
                                      len(y_right) * self._gini(y_right)) / len(y)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
                    
        return best_feature, best_threshold
    
    def fit(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        
        if (self.max_depth is not None and depth >= self.max_depth) or n_classes == 1:
            self.tree = {'prediction': np.bincount(y).argmax()}
            return self
            
        feature, threshold = self._best_split(X, y)
        
        if feature is None:
            self.tree = {'prediction': np.bincount(y).argmax()}
            return self
            
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        
        left_tree = DecisionTree(self.max_depth)
        right_tree = DecisionTree(self.max_depth)
        
        left_tree.fit(X_left, y_left, depth + 1)
        right_tree.fit(X_right, y_right, depth + 1)
        
        self.tree = {
            'feature': feature,
            'threshold': threshold,
            'left': left_tree,
            'right': right_tree
        }
        
        return self
    
    def predict_single(self, x):
        tree = self.tree
        while 'prediction' not in tree:
            if x[tree['feature']] <= tree['threshold']:
                tree = tree['left'].tree
            else:
                tree = tree['right'].tree
        return tree['prediction']
    
    def predict(self, X):
        return np.array([self.predict_single(x) for x in X])

# Scale the data and prepare labels
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_labels = np.argmax(y, axis=1)

# Split the scaled data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_labels, test_size=0.2, random_state=42)

# Train Decision Tree model
model = DecisionTree(max_depth=3)  # Reduced depth for better visualization
model.fit(X_train, y_train)

# Get predictions
y_pred = model.predict(X_test)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Decision Tree Accuracy: {accuracy:.4f}")

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Decision Tree Accuracy: 1.0000


In [34]:
class GaussianNaiveBayes:
    def __init__(self):
        self.classes = None
        self.priors = None
        self.means = None
        self.vars = None
        
    def _gaussian_pdf(self, x, mean, var):
        return np.exp(-(x - mean)**2 / (2 * var)) / np.sqrt(2 * np.pi * var)
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        # Calculate class priors
        self.priors = np.zeros(n_classes)
        self.means = np.zeros((n_classes, n_features))
        self.vars = np.zeros((n_classes, n_features))
        
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[i] = len(X_c) / n_samples
            self.means[i] = X_c.mean(axis=0)
            self.vars[i] = X_c.var(axis=0) + 1e-9  # Add small value for numerical stability
        
        return self
    
    def predict_proba(self, X):
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        probs = np.zeros((n_samples, n_classes))
        
        for i in range(n_classes):
            prior = np.log(self.priors[i])
            likelihood = np.sum(np.log(self._gaussian_pdf(X, self.means[i], self.vars[i])), axis=1)
            probs[:, i] = prior + likelihood
            
        # Convert log probabilities to probabilities
        probs = np.exp(probs - probs.max(axis=1)[:, np.newaxis])
        return probs / probs.sum(axis=1)[:, np.newaxis]
    
    def predict(self, X):
        return self.classes[np.argmax(self.predict_proba(X), axis=1)]

# Scale the data and prepare labels
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_labels = np.argmax(y, axis=1)

# Split the scaled data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_labels, test_size=0.2, random_state=42)

# Train Naive Bayes model
model = GaussianNaiveBayes()
model.fit(X_train, y_train)

# Get predictions and probabilities
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Naive Bayes Accuracy: {accuracy:.4f}")


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Naive Bayes Accuracy: 1.0000


In [35]:
class LinearDiscriminantAnalysis:
    def __init__(self, n_components=None):
        self.n_components = n_components
        self.classes = None
        self.means = None
        self.weights = None
        self.covariance = None
        self.priors = None
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        # Calculate class means and priors
        self.means = []
        self.priors = []
        for i, cls in enumerate(self.classes):
            X_c = X[y == cls]
            self.means.append(X_c.mean(axis=0))
            self.priors.append(len(X_c) / n_samples)
        
        self.means = np.array(self.means)
        self.priors = np.array(self.priors)
        
        # Calculate within-class covariance matrix
        self.covariance = np.zeros((n_features, n_features))
        for i, cls in enumerate(self.classes):
            X_c = X[y == cls]
            X_c = X_c - self.means[i]
            self.covariance += X_c.T @ X_c
        
        self.covariance /= n_samples
        
        # Calculate weights using inverse covariance
        try:
            inv_covariance = np.linalg.inv(self.covariance)
        except np.linalg.LinAlgError:
            # Add small identity matrix if singular
            inv_covariance = np.linalg.inv(self.covariance + 1e-4 * np.eye(n_features))
            
        self.weights = self.means @ inv_covariance
        
        return self
    
    def predict_proba(self, X):
        discriminant = X @ self.weights.T
        discriminant += np.log(self.priors) - 0.5 * np.sum(self.means @ self.weights.T, axis=1)
        
        # Convert to probabilities using softmax
        exp_disc = np.exp(discriminant - discriminant.max(axis=1)[:, np.newaxis])
        probs = exp_disc / exp_disc.sum(axis=1)[:, np.newaxis]
        return probs
    
    def predict(self, X):
        return self.classes[np.argmax(self.predict_proba(X), axis=1)]

# Scale the data and prepare labels
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_labels = np.argmax(y, axis=1)

# Split the scaled data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_labels, test_size=0.2, random_state=42)

# Train LDA model
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

# Get predictions and probabilities
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"LDA Accuracy: {accuracy:.4f}")

[2 0 2 2 2 0 2 2 2 2 2 0 0 0 0 2 2 2 2 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
LDA Accuracy: 0.7000


In [37]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.tree = {}
        
    def _mse_split(self, X, y):
        best_score = float('inf')
        best_split = None
        
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                if left_mask.sum() == 0 or left_mask.sum() == len(y):
                    continue
                    
                left_pred = y[left_mask].mean()
                right_pred = y[~left_mask].mean()
                score = np.sum((y[left_mask] - left_pred)**2) + np.sum((y[~left_mask] - right_pred)**2)
                
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold, left_pred, right_pred)
                    
        return best_split
    
    def fit(self, X, y, depth=0):
        if depth >= self.max_depth or len(np.unique(y)) == 1:
            self.tree = {"prediction": y.mean()}
            return
            
        split = self._mse_split(X, y)
        if split is None:
            self.tree = {"prediction": y.mean()}
            return
            
        feature, threshold, left_pred, right_pred = split
        left_mask = X[:, feature] <= threshold
        
        self.tree = {
            "feature": feature,
            "threshold": threshold,
            "left": DecisionTreeRegressor(self.max_depth),
            "right": DecisionTreeRegressor(self.max_depth)
        }
        
        self.tree["left"].fit(X[left_mask], y[left_mask], depth + 1)
        self.tree["right"].fit(X[~left_mask], y[~left_mask], depth + 1)
    
    def predict(self, X):
        return np.array([self._predict_single(x) for x in X])
    
    def _predict_single(self, x):
        node = self.tree
        while "prediction" not in node:
            if x[node["feature"]] <= node["threshold"]:
                node = node["left"].tree
            else:
                node = node["right"].tree
        return node["prediction"]

class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.classes_ = None
        
    def _softmax(self, x):
        exp_x = np.exp(x - x.max(axis=1, keepdims=True))
        return exp_x / exp_x.sum(axis=1, keepdims=True)
    
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        y_encoded = np.eye(n_classes)[y]
        
        # Initialize predictions with zeros
        F = np.zeros((X.shape[0], n_classes))
        
        for m in range(self.n_estimators):
            # Calculate gradients
            P = self._softmax(F)
            gradients = P - y_encoded
            
            # Train trees for each class
            trees_m = []
            for k in range(n_classes):
                tree = DecisionTreeRegressor(max_depth=self.max_depth)
                tree.fit(X, -gradients[:, k])
                trees_m.append(tree)
            
            # Update F with predictions
            for k in range(n_classes):
                F[:, k] += self.learning_rate * trees_m[k].predict(X)
            
            self.trees.append(trees_m)
        
        return self
    
    def predict_proba(self, X):
        # Sum up predictions from all trees
        F = np.zeros((X.shape[0], len(self.classes_)))
        for trees_m in self.trees:
            for k, tree in enumerate(trees_m):
                F[:, k] += self.learning_rate * tree.predict(X)
        return self._softmax(F)
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

# Scale the data and prepare labels
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_labels = np.argmax(y, axis=1)

# Split the scaled data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_labels, test_size=0.2, random_state=42)

# Train Gradient Boosting model
model = GradientBoostingClassifier(n_estimators=50, learning_rate=0.1)
model.fit(X_train, y_train)

# Get predictions and probabilities
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

print(y_pred)
print(y_test_labels)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Gradient Boosting Accuracy: {accuracy:.4f}")

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Gradient Boosting Accuracy: 1.0000
